##  Setup

In [1]:
include("../MyReverseDiff.jl")
include("../MyEmbedding.jl")
include("../MyMlp.jl")

using .MyReverseDiff
using .MyEmbedding
using .MyMlp
using Printf
using LinearAlgebra
using Random

##  Przygotowanie danych testowych

In [2]:
# Mini słownik (bardzo mały dla prostoty)
vocab = ["good", "bad", "movie", "very", "<PAD>"]
vocab_size = length(vocab)
embedding_dim = 3  # Małe embedding dla łatwych obliczeń
sequence_length = 4  # Długość sekwencji
num_of_words = 5  # Liczba słów w słowniku

println("Słownik:")
for (i, word) in enumerate(vocab)
    println("$i: $word")
end

println("\nParametry:")
println("- Rozmiar słownika: $vocab_size")
println("- Wymiar embedding: $embedding_dim")
println("- Długość sekwencji: $sequence_length")
println("- Liczba słów w słowniku: $num_of_words")

# Sekwencje tekstowe jako indeksy
# Sekwencja 1: "very good movie <PAD>" → [5, 2, 4, 1] → pozytywna (klasa 1)
# Sekwencja 2: "very bad movie <PAD>" → [5, 3, 4, 1] → negatywna (klasa 0)

sequence_1 = Float32[4, 1, 3, 5]  # very good movie <PAD>
sequence_2 = Float32[4, 2, 3, 5]  # very bad movie <PAD>

# Konwertuj na format batch (sequence_length, batch_size)
X_batch = zeros(Float32, sequence_length, 2)
X_batch[:, 1] = sequence_1
X_batch[:, 2] = sequence_2

# Etykiety: sekwencja 1 → pozytywna (1), sekwencja 2 → negatywna (0)
y_batch = Float32[1.0 0.0]  # (1, 2) 

println("Sekwencja 1 (pozytywna): ", [vocab[Int(i)] for i in sequence_1])
println("Indeksy: ", sequence_1)
println("\nSekwencja 2 (negatywna): ", [vocab[Int(i)] for i in sequence_2])
println("Indeksy: ", sequence_2)
println("\nShape danych X: ", size(X_batch))
println("Etykiety y: ", y_batch)
println("Shape etykiet: ", size(y_batch))

# Embedding weights - każde słowo ma swój unikalny wektor
embedding_weights = Float32[
    1.0  -1.0   0.5  0.0 0.0;   # wymiar 1: <PAD>, good, bad, movie, very
    0.5   0.5   0.0  1.0 0.0;   # wymiar 2
    1.0  -1.0   1.0  0.0 0.0    # wymiar 3
]

for (i, word) in enumerate(vocab)
    vec = embedding_weights[:, i]
    println("$word: $vec")
end

Słownik:
1: good
2: bad
3: movie
4: very
5: <PAD>

Parametry:
- Rozmiar słownika: 5
- Wymiar embedding: 3
- Długość sekwencji: 4
- Liczba słów w słowniku: 5
Sekwencja 1 (pozytywna): ["very", "good", "movie", "<PAD>"]
Indeksy: Float32[4.0, 1.0, 3.0, 5.0]

Sekwencja 2 (negatywna): ["very", "bad", "movie", "<PAD>"]
Indeksy: Float32[4.0, 2.0, 3.0, 5.0]

Shape danych X: (4, 2)
Etykiety y: Float32[1.0 0.0]
Shape etykiet: (1, 2)
good: Float32[1.0, 0.5, 1.0]
bad: Float32[-1.0, 0.5, -1.0]
movie: Float32[0.5, 0.0, 1.0]
very: Float32[0.0, 1.0, 0.0]
<PAD>: Float32[0.0, 0.0, 0.0]


##  Warstwa embedding - MyCNN Forward

In [3]:
embedding_layer = MyMlp.Embedding(embedding_weights; name="my_embedding_layer")
indices_node = MyReverseDiff.Constant(X_batch)
embedding_output_node = embedding_layer(indices_node)
order = MyReverseDiff.topological_sort(embedding_output_node)
forward_pass = MyReverseDiff.forward!(order)

3×4×2 Array{Float32, 3}:
[:, :, 1] =
 0.0  1.0  0.5  0.0
 1.0  0.5  0.0  0.0
 0.0  1.0  1.0  0.0

[:, :, 2] =
 0.0  -1.0  0.5  0.0
 1.0   0.5  0.0  0.0
 0.0  -1.0  1.0  0.0

##  Warstwa embedding - MyCNN Backward

In [4]:
output = MyReverseDiff.backward!(order)

##  Warstwa Dense 3D - MyCNN Forward

In [5]:
# Chcemy transformować każdy wektor o długości `embedding_dim`
# na nowy wektor o długości `dense_output_dim`.
dense_output_dim = 2 # Załóżmy, że chcemy uzyskać wektory 5-wymiarowe
# Predefiniowane wagi i bias (te same)
custom_weights = Float32[
    10.0  0.0  -1.0;
     0.0  1.0   2.0
]
custom_bias = Float32[1.0; -2.0]  # 2-elementowy wektor kolumnowy
custom_bias_matrix = reshape(custom_bias, :, 1)  # => 2×1 macierz


2×1 Matrix{Float32}:
  1.0
 -2.0

In [6]:
dense_layer = MyMlp.Dense3D(embedding_dim, dense_output_dim, MyMlp.relu; name="my_dense_layer")
dense_input = MyReverseDiff.Constant(forward_pass)
dense_output_node = dense_layer(dense_input)
dense_order = MyReverseDiff.topological_sort(dense_output_node)

dense_order[2].output = custom_weights
dense_order[3].output = custom_bias_matrix


2×1 Matrix{Float32}:
  1.0
 -2.0

In [7]:
dense_forward_pass = MyReverseDiff.forward!(dense_order)

2×4×2 Array{Float32, 3}:
[:, :, 1] =
  1.0  10.0  5.0   1.0
 -0.0   0.5  0.0  -0.0

[:, :, 2] =
  1.0  -0.0  5.0   1.0
 -0.0  -0.0  0.0  -0.0

##  Dense 3D - MyCNN Backward

In [8]:
MyReverseDiff.backward!(dense_order)

##  Max Pool (4,) CNN Forward

In [32]:
X = dense_forward_pass

2×4×2 Array{Float32, 3}:
[:, :, 1] =
  1.0  10.0  5.0   1.0
 -0.0   0.5  0.0  -0.0

[:, :, 2] =
  1.0  -0.0  5.0   1.0
 -0.0  -0.0  0.0  -0.0

In [33]:
max_pooling_layer = MyMlp.PoolingBlock(4)
pooling_input = MyReverseDiff.Constant(X)

mat = [4.0f0]  # tworzy 1×1 macierz
mat = reshape(mat, 1, 1)  # zapewnia, że jest to macierz 1×1
pooling_window = MyReverseDiff.Constant(mat)  # Okno poolingowe o rozmiarze 4

pooling_output_node = max_pool(pooling_input, pooling_window)
pooling_order = MyReverseDiff.topological_sort(pooling_output_node)

3-element Vector{GraphNode}:
 const Float32[1.0 10.0 5.0 1.0; -0.0 0.5 0.0 -0.0;;; 1.0 -0.0 5.0 1.0; -0.0 -0.0 0.0 -0.0]
 const Float32[4.0;;]
 op.?(typeof(max_pool))

In [34]:
pooling_forward_pass = MyReverseDiff.forward!(pooling_order)

0×4×2 Array{Float32, 3}